In [ ]:
#작업을 진행하기 전에, DB이름 : registration으로 먼저 만들어주고 DB 유저를 생성해주세요
#1. Student 테이블 만들기
#mysql에 접속이 안되면 본인 로컬 mysql에 계정 권한을 주고 다시 시도해주세요
import scipy.io
import csv
import pymysql

db_user = 'jhyun1' #접속할 DB 이름
db_host = 'localhost' #접속할 DB 호스트
db_password = 's852130'#DB 비밀번호
db_charset = 'utf8'
db_name = 'registration'

conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
curs = conn.cursor()

CREATE_STUDENT_TABLE = "create table student("\
"student_id varchar(12) NOT NULL UNIQUE KEY,"\
"password varchar(45) NOT NULL,"\
"name varchar(45) NOT NULL,"\
"department varchar(45) NOT NULL,"\
"term varchar(45),"\
"division varchar(45),"\
"PRIMARY KEY(student_id)"\
") ENGINE = InnoDB;"
print(CREATE_STUDENT_TABLE)

AffectedRows = curs.execute(CREATE_STUDENT_TABLE)
conn.commit()
print('Number of affected rows: %d' %(AffectedRows))
conn.close()

In [ ]:
#2. 학정정보.csv에서 데이터를 읽어와 테이블에 넣기
conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
curs = conn.cursor()
with open('./학정정보.csv', 'rt', encoding='UTF8') as f:
    csvReader = csv.reader(f)
    i = 0
    for row in csvReader:
        if i == 0: #첫 행은 거름
            i += 1
            continue
        student_id = (row[0]) #학번
        password = ('q1w2e3r4') #비밀번호 값은 임의로 넣음
        name = row[1]    #이름
        department = ('컴퓨터공학과') #학과
        term = (row[2]) #학적사항
        division = (row[3]) #분반
        print(student_id, password, name, department, term, division)
        sql = """insert into student(student_id, password, name, department, term, division)"""\
        """values(%s, %s, %s, %s, %s, %s);"""
        if len(division) == 0:
            sql = """insert into student(student_id, password, name, department, term)"""\
        """values(%s, %s, %s, %s, %s);"""
            curs.execute(sql, (student_id, password, name, department, term))
        else:
            curs.execute(sql, (student_id, password, name, department, term, division))
    
    conn.commit()
    conn.close()

In [ ]:
CREATE_SUBJECT_TABLE = "CREATE TABLE subject("\
"subject_id varchar(100) not null,"\
"term varchar(10) not null,"\
"subject_name varchar(100) not null,"\
"division varchar(100) DEFAULT NULL,"\
"is_major tinyint(1) NOT NULL,"\
"class_hours INT NOT NULL,"\
"professor_name varchar(100) DEFAULT NULL,"\
"day1 varchar(10) DEFAULT NULL,"\
"time1 varchar(10) DEFAULT NULL,"\
"day2 varchar(10) DEFAULT NULL,"\
"time2 varchar(10) DEFAULT NULL,"\
"people INT,"\
"PRIMARY KEY(subject_id, term)"\
") ENGINE = InnoDB;"
print(CREATE_SUBJECT_TABLE)
try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_SUBJECT_TABLE)
    conn.commit()
    
    with open('./개설 교과목 정보.csv', 'rt', encoding='UTF8') as f:
        csvReader = csv.reader(f)
        i = 0
        for row in csvReader:
            if i == 0:
                i += 1
                continue
            sql = """insert into subject(subject_id, term, subject_name, division, is_major, class_hours, professor_name, day1, time1, day2, time2, people)"""\
            """values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
            subject_id = row[0]
            term = row[1]
            subject_name = row[2]
            division = row[3] if len(row[3]) != 0 else ''
            is_major = 1 if row[4] == '전공' else 0
            class_hours = row[5]
            professor_name = row[6] if len(row[6]) != 0 else ''
            day1 = row[7] if len(row[7]) != 0 else ''
            time1 = row[8] if len(row[8]) != 0 else ''
            day2 = row[9] if len(row[9]) != 0 else ''
            time2 = row[10] if len(row[10]) != 0 else ''
            people = row[11]
            curs.execute(sql, (subject_id, term, subject_name, division, is_major, class_hours, professor_name, day1, time1, day2, time2, people))
        conn.commit()
finally:
    conn.close()

In [ ]:
CREATE_SUBJECT_EVAL_TABLE = "CREATE TABLE subject_eval("\
"subject_id VARCHAR(100) NOT NULL,"\
"term VARCHAR(10) NOT NULL,"\
"subject_eval varchar(2000) NOT NULL,"\
"CONSTRAINT FK_ID_TERM FOREIGN KEY(subject_id, term) REFERENCES subject(subject_id, term) ON UPDATE CASCADE"\
") ENGINE = InnoDB;"
print(CREATE_SUBJECT_EVAL_TABLE)
try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_SUBJECT_EVAL_TABLE)
    conn.commit()
    
    with open("./강의평가.csv", "rt", encoding="UTF8") as f:
        csvReader = csv.reader(f)
        i = 0
        for row in csvReader:
            if i == 0:
                i += 1
                continue
            sql = """insert into subject_eval(subject_id, term, subject_eval) values(%s, %s, %s)"""
            subject_id = row[0]
            term = row[1]
            subject_eval = row[2]
            curs.execute(sql, (subject_id, term, subject_eval))
    conn.commit()
finally:
    conn.close()